In [ ]:
!qiime cscs cscs --p-css-edges gfop_edges_translated.txt --i-features gfop_merged-feature-table.qza  --p-normalization --o-distance-matrix gfop_cscs_distance_matrix.qza --p-cpus 5 --p-chunk 1000

In [24]:
import qiime2.plugins.diversity.actions as q2_diversity
import qiime2.plugins.feature_table.actions as q2_feature_table
from qiime2 import Artifact
from qiime2 import Metadata
import qiime2 as q2
import pandas as pd
from skbio.stats.distance import permanova
from skbio import DistanceMatrix, OrdinationResults





In [16]:
def load_mf(fn, index='sample_name', sep='\t'):
    _df = pd.read_csv(fn, sep=sep, dtype='str', na_values=[], keep_default_na=False)
    _df.set_index(index, inplace=True)
    return _df

In [46]:
#Load distances
cscs_dist = q2.Artifact.load("gfop_cscs_distance_matrix.qza")
cscs_dist_dm = cscs_dist.view(DistanceMatrix)
qe_dist =  q2.Artifact.load("gfop_distance-matrix.qza")
qe_dist_dm = qe_dist.view(DistanceMatrix)

#Load and filter mapping
mapping = load_mf("gfop_modified_qiime2_metadata.tsv")
mappingf = mapping.loc[list(cscs_dist_dm.ids)]

# Test sample_type_group1_labels
sample_type_group1_labels = mappingf["sample_type_group1"].values
print(permanova(cscs_dist_dm, sample_type_group1_labels))
print(permanova(qe_dist_dm, sample_type_group1_labels))

# Test sample_type_group2_labels
sample_type_group2_labels = mappingf["sample_type_group2"].values
print(permanova(cscs_dist_dm, sample_type_group2_labels))
print(permanova(qe_dist_dm, sample_type_group2_labels))

# Test sample_type_group3_labels
sample_type_group3_labels = mappingf["sample_type_group3"].values
print(permanova(cscs_dist_dm, sample_type_group3_labels))
print(permanova(qe_dist_dm, sample_type_group3_labels))

# Test sample_type_group4_labels
sample_type_group4_labels = mappingf["sample_type_group4"].values
print(permanova(cscs_dist_dm, sample_type_group4_labels))
print(permanova(qe_dist_dm, sample_type_group4_labels))


method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                  5
test statistic              4.63819
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                  5
test statistic              5.34768
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                  6
test statistic              4.53509
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups   

In [83]:
#CSCS analysis
import itertools
import matplotlib.pyplot as plt

def load_mf_qe(fn, index='#SampleID', sep='\t'):
    _df = pd.read_csv(fn, sep=sep, dtype='str', na_values=[], keep_default_na=False)
    _df.set_index(index, inplace=True)
    return _df


def summarize_distances(dm, metric, metadata, grouping='ATTRIBUTE_Proportions', 
                        catdiff='ATTRIBUTE_Experiment', comparison='across'):
    
    if comparison not in {'across', 'within'}:
        raise ValueError('Unsupported comparison style: %s' % comparison)
    
    groups = metadata[grouping].unique()
    pair_dist = {}
    
    for val in groups:
        # exclude blanks
        if val == '0_0_0_0':
            continue
            
        # subset to only the samples in this grouping category
        sampls = set(dm.ids).intersection(metadata[metadata[grouping] == val].index)

        if comparison == 'across':
            subset_md = metadata.loc[sampls] # only samples with grouping = val
            categories = subset_md[catdiff].unique() # catdiffs per grouping = val
            cat_samp_list = [] # list of samples per catdiff, len of this list = num_categories

            for cat in categories:
                cat_samp_list.append(list(subset_md[subset_md[catdiff] ==  cat].index))
            pair_cats = list(itertools.combinations(range(len(categories)), 2)) # pairs of categories

            for pairs in pair_cats:
                res = list(itertools.product(*[cat_samp_list[pairs[0]],cat_samp_list[pairs[1]]])) # all pairs across categories
                for samp_pair in res:
                    pair_dist[samp_pair] = [dm[samp_pair], val, metric, 'across']
        elif comparison == 'within':
            
            for _, _df in metadata.loc[sampls].groupby(catdiff):
            
                res = list(itertools.combinations(_df.index, 2)) # all pairs for a given proportion
                
                for samp_pair in res:
                    pair_dist[samp_pair] = [dm[samp_pair], val, metric, 'within']
            

    return pd.DataFrame.from_dict(data=pair_dist, orient='index',
                                  columns=['distance', 'grouping', 'metric', 'comparison'])

cscs_dist = q2.Artifact.load("data/cscs_distance_matrix.qza")
dm = cscs_dist.view(DistanceMatrix)

mapping_path = 'data/qe_qert_qiime2_metadata.tsv'
_mapping = load_mf(mapping_path, 'sample_name')
blanks = set(_mapping.query('ATTRIBUTE_Sample_type != "Sample"').index)
    
samples = set(_mapping.index) - set(blanks)

print(' blanks: %d, samples: %d' % ( len(blanks), len(samples)))

    # Add a new column that describes if the sample in this dataset is a blank.

_mapping['is_blank'] = "False"
_mapping.loc[blanks, 'is_blank'] = "True"

_mapping.replace(to_replace='', value='NA', inplace=True)
_mapping = _mapping.loc[samples]

labels = [_mapping.loc[sname, 'ATTRIBUTE_Experiment'] for sname in dm.ids]

permanova(dm, labels)

 blanks: 85, samples: 486
(162, 162)
(162, 162)


method name               PERMANOVA
test statistic name        pseudo-F
sample size                     162
number of groups                  2
test statistic              13.5544
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object

In [ ]:
pcoa = q2_diversity.pcoa(cscs_dist)[0].view(OrdinationResults)

colors = {'C18': 'red',
          'C18-RTshift': 'blue'}
benchmark = 'ATTRIBUTE_Experiment'
distances = []
distances.append(summarize_distances(dm, "CSCS" + ' across',
                                             _mapping, comparison='across', catdiff=benchmark))
distances.append(summarize_distances(dm, "CSCS" + ' within',
                                             _mapping, comparison='within', catdiff=benchmark))

fig, axes = plt.subplots(nrows=1, ncols=1, sharex=False, sharey=False, figsize=(20, 5))
ax = axes#[0]
ax.set_aspect(1, adjustable='box', anchor='SW')
ax.set_title("CSCS")

pc = pcoa


for proportion, _df in _mapping.groupby('ATTRIBUTE_Proportions'):
    for triplicate, __df in _df.groupby('Triplicate_number'):
        for u, v in itertools.combinations(__df.index, 2):
            print(u,v)
            print(_mapping.shape, )
            
            #print(pc.samples)
                    # use the underlying numpy array so that matplotlib won't take labels
                    # from the columns/index
            _pc = pc.samples.loc[[u, v]].values
            ax.plot(_pc[:, 0], _pc[:, 1], c='gray', alpha=0.1, zorder=-999, label=None)
            
            